# Fully connected network

https://adamharley.com/nn_vis/mlp/3d.html

# Membuat Neural Network menggunakan pytorch

In [53]:
## Batch training

# Convolutional Neural Network

https://adamharley.com/nn_vis/cnn/3d.html